In [1]:
DATASET_FILE_1M20PHS = "./dataset/1m20phs.csv"

In [5]:
import polars as pl
dataset_1m20phs = pl.read_csv(DATASET_FILE_1M20PHS)

In [8]:
# File to store the achieved load and pps for 1M20PHS workload

column_names = ['offered_load_pps', 'achieved_load_pps', 'p99', 'pinning_limit_2mb_pages', 'segment_size_2mb_pages', 'system']
dataset_columns_1m20phs = dataset_1m20phs.select(column_names)

In [51]:
from typing import Optional, Union


def get_unique_column_entries(dataframe: pl.DataFrame, column_name):
    return dataframe[column_name].unique()

def convert_to_plotting_dict(dataframe: pl.DataFrame):
    system_names = dataframe[['system']].unique().rows()
    offered_load = dataframe[['offered_load_pps']].unique().rows()
    pinning_limit = dataframe[['pinning_limit_2mb_pages']].unique().rows()
    plotting_dict = {}
    for system in system_names:
        if system[0] not in plotting_dict.keys():
            plotting_dict[system[0]] = {}
        for load in offered_load:
            if load[0] not in plotting_dict[system[0]].keys():
                plotting_dict[system[0]][load[0]]={}

            for pin_limit in pinning_limit:
                if pin_limit[0] not in plotting_dict[system[0]][load[0]].keys():
                    plotting_dict[system[0]][load[0]][pin_limit[0]] = []

    for k in plotting_dict.keys():
        for l in plotting_dict[k].keys():
            for m in plotting_dict[k][l].keys():
                for data in dataframe:
                    if data['system'] == k and data['offered_load_pps'] == l and data['pinning_limit_2mb_pages'] == m:
                        plotting_dict[k][l][m].append((data['achieved_load_pps'], data['p99']))
    
    return plotting_dict
                    
segment_size_grouped_dataframe = dataset_columns_1m20phs.group_by('segment_size_2mb_pages')
for df in segment_size_grouped_dataframe:
    conv = convert_to_plotting_dict(df[1])
    print(conv)
    break

TypeError: cannot use `__getitem__` on Series of dtype Int64 with argument 'system' of type 'str'

In [30]:
system_names = list(get_unique_column_entries(dataset_columns_1m20phs, ['system']))
system_names

[shape: (3,)
 Series: 'system' [str]
 [
 	"zcc_cornflakes…
 	"vanilla_cornfl…
 	"cornflakes_cop…
 ]]

In [31]:
systems_grouped_dataframe = dataset_columns_1m20phs.group_by('system')
for df in systems_grouped_dataframe:
    print(df[1]['segment_size_2mb_pages'].unique())

shape: (3,)
Series: 'segment_size_2mb_pages' [i64]
[
	16
	32
	64
]
shape: (3,)
Series: 'segment_size_2mb_pages' [i64]
[
	16
	32
	64
]
shape: (3,)
Series: 'segment_size_2mb_pages' [i64]
[
	16
	32
	64
]


In [16]:
segment_size_grouped_dataframe = dataset_columns_1m20phs.group_by('segment_size_2mb_pages')
for df in segment_size_grouped_dataframe:
    conv = convert_to_plotting_dict(df[1])
    print(conv)
    break

TypeError: unhashable type: 'Series'